In [45]:
from astroquery.gaia import Gaia
import numpy as np
import pandas as pd

## 1.- Importación del dataset

In [ ]:
Gaia.login(user = "", password="")

job = Gaia.launch_job_async("""
-- GIANTS
SELECT TOP 500
  ra, dec, parallax, parallax_error,
  phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag,
  ruwe, phot_bp_rp_excess_factor, phot_g_mean_flux,
  astrometric_excess_noise, bp_rp, bp_g, g_rp,random_index 
FROM gaiadr3.gaia_source
WHERE parallax_over_error > 10
  AND ruwe < 1.4
  AND (phot_bp_mean_mag - phot_rp_mean_mag) >= 0.8
  AND (phot_g_mean_mag - 10 + 5*LOG10(parallax)) <= 2.5
ORDER BY random_index

UNION ALL

-- WDs
SELECT TOP 500
  ra, dec, parallax, parallax_error,
  phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag,
  ruwe, phot_bp_rp_excess_factor, phot_g_mean_flux,
  astrometric_excess_noise, bp_rp, bp_g, g_rp, random_index
FROM gaiadr3.gaia_source
WHERE parallax_over_error > 10
  AND ruwe < 1.4
  AND (phot_bp_mean_mag - phot_rp_mean_mag) BETWEEN -0.5 AND 1.8
  AND (phot_g_mean_mag - 10 + 5*LOG10(parallax)) >= 10
ORDER BY random_index

UNION ALL

-- MS
SELECT TOP 500
  ra, dec, parallax, parallax_error,
  phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag,
  ruwe, phot_bp_rp_excess_factor, phot_g_mean_flux,
  astrometric_excess_noise, bp_rp, bp_g, g_rp, random_index
FROM gaiadr3.gaia_source
WHERE parallax_over_error > 10
  AND ruwe < 1.4
  AND NOT (
    ((phot_bp_mean_mag - phot_rp_mean_mag) >= 0.8
     AND (phot_g_mean_mag - 10 + 5*LOG10(parallax)) <= 2.5)
    OR
    ((phot_bp_mean_mag - phot_rp_mean_mag) BETWEEN -0.5 AND 1.8
     AND (phot_g_mean_mag - 10 + 5*LOG10(parallax)) >= 10)
  )
ORDER BY random_index


""")

results = job.get_results()
df = results.to_pandas()
df.head()

INFO: Login to gaia TAP server [astroquery.gaia.core]
401 Error 401:
<!doctype html><html lang="en"><head><title>HTTP Status 401 – Unauthorized</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 401 – Unauthorized</h1><hr class="line" /><p><b>Type</b> Status Report</p><p><b>Message</b> Bad Credentials</p><p><b>Description</b> The request has not been applied to the target resource because it lacks valid authentication credentials for that resource.</p><hr class="line" /><h3>Apache Tomcat/9.0.102</h3></body></html>


ERROR: Error logging in TAP server [astroquery.gaia.core]


INFO: Query finished. [astroquery.utils.tap.core]


,ra,dec,parallax,parallax_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,ruwe,phot_bp_rp_excess_factor,phot_g_mean_flux,astrometric_excess_noise,bp_rp,bp_g,g_rp,random_index
0,80.027527,-49.785186,0.468424,0.012727,13.993294,14.509561,13.325885,0.971992,1.229128,47602.451754,0.000000,1.183676,0.516267,0.667409,329
1,19.826662,33.865898,0.344142,0.024055,14.778735,15.179917,14.204826,0.943659,1.215310,23091.534139,0.000000,0.975091,0.401182,0.573909,532
2,64.880506,-31.088960,0.849927,0.018639,11.515061,12.020891,10.861386,1.368054,1.223699,466575.903749,0.178768,1.159505,0.505830,0.653675,616
3,221.391547,-52.847784,0.183334,0.017172,13.821015,15.180682,12.687241,0.999491,1.403276,55787.957829,0.000000,2.493442,1.359667,1.133775,643
4,88.212522,42.019391,0.717614,0.013189,13.168218,13.620908,12.545147,0.958131,1.225167,101779.386976,0.000000,1.075761,0.452690,0.623071,698


## 2.- Cálculo de BP_RP, M_G y clásificación

In [47]:
df["BP_RP"] = df["phot_bp_mean_mag"] - df["phot_rp_mean_mag"]
d_pc = 1000.0 / df["parallax"]   # distancia en parsecs (parallax en mas)
df["M_G"] = df["phot_g_mean_mag"] + 5 - 5*np.log10(d_pc)

df.head()

,ra,dec,parallax,parallax_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,ruwe,phot_bp_rp_excess_factor,phot_g_mean_flux,astrometric_excess_noise,bp_rp,bp_g,g_rp,random_index,BP_RP,M_G
0,80.027527,-49.785186,0.468424,0.012727,13.993294,14.509561,13.325885,0.971992,1.229128,47602.451754,0.000000,1.183676,0.516267,0.667409,329,1.183676,2.346488
1,19.826662,33.865898,0.344142,0.024055,14.778735,15.179917,14.204826,0.943659,1.215310,23091.534139,0.000000,0.975091,0.401182,0.573909,532,0.975091,2.462424
2,64.880506,-31.088960,0.849927,0.018639,11.515061,12.020891,10.861386,1.368054,1.223699,466575.903749,0.178768,1.159505,0.505830,0.653675,616,1.159505,1.161969
3,221.391547,-52.847784,0.183334,0.017172,13.821015,15.180682,12.687241,0.999491,1.403276,55787.957829,0.000000,2.493442,1.359667,1.133775,643,2.493442,0.137227
4,88.212522,42.019391,0.717614,0.013189,13.168218,13.620908,12.545147,0.958131,1.225167,101779.386976,0.000000,1.075761,0.452690,0.623071,698,1.075761,2.447671


In [48]:
def label_star(row):
    # Enanas blancas: azules y muy débiles
    if (-0.5 <= row.BP_RP <= 1.8) and (row.M_G >= 10):
        return "WD"      # White Dwarf
    # Gigantes: rojas y muy brillantes
    elif (row.BP_RP >= 0.8) and (row.M_G <= 2.5):
        return "Giant"
    # Secuencia principal: todo lo demás
    else:
        return "MS"      # Main Sequence

In [49]:
df["target"] = df.apply(label_star, axis=1)
print(df["target"].value_counts())
df.head()

target
Giant    500
WD       500
MS       500
Name: count, dtype: int64


,ra,dec,parallax,parallax_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,ruwe,phot_bp_rp_excess_factor,phot_g_mean_flux,astrometric_excess_noise,bp_rp,bp_g,g_rp,random_index,BP_RP,M_G,target
0,80.027527,-49.785186,0.468424,0.012727,13.993294,14.509561,13.325885,0.971992,1.229128,47602.451754,0.000000,1.183676,0.516267,0.667409,329,1.183676,2.346488,Giant
1,19.826662,33.865898,0.344142,0.024055,14.778735,15.179917,14.204826,0.943659,1.215310,23091.534139,0.000000,0.975091,0.401182,0.573909,532,0.975091,2.462424,Giant
2,64.880506,-31.088960,0.849927,0.018639,11.515061,12.020891,10.861386,1.368054,1.223699,466575.903749,0.178768,1.159505,0.505830,0.653675,616,1.159505,1.161969,Giant
3,221.391547,-52.847784,0.183334,0.017172,13.821015,15.180682,12.687241,0.999491,1.403276,55787.957829,0.000000,2.493442,1.359667,1.133775,643,2.493442,0.137227,Giant
4,88.212522,42.019391,0.717614,0.013189,13.168218,13.620908,12.545147,0.958131,1.225167,101779.386976,0.000000,1.075761,0.452690,0.623071,698,1.075761,2.447671,Giant


# Eliminación de NaN's

In [50]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Features (X) y target (y)
X = df.drop(columns=["target"])
y = df["target"]

# Imputar valores NaN con la media de cada columna
imputer = SimpleImputer(strategy="mean")
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

print("NaNs antes:", X.isna().sum().sum())
print("NaNs después:", X_imputed.isna().sum().sum())

NaNs antes: 14
NaNs después: 0


# Separación del target, split y entrenamiento con RF y REF

In [51]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# Ejemplo: define tus features (agrega todas las que quieras probar)
features = df.columns.difference(["target","random_index"])  # todas menos estas
X = df[features].copy()
y = df["target"].copy()           # "MS", "Giant", "WD"

# codifica y para métricas como ROC multi-clase
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [52]:
from sklearn.model_selection import train_test_split

# Primero sacamos el conjunto de prueba (20%)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y_enc, test_size=0.2, stratify=y_enc, random_state=42
)

# De lo que quedó (80%), hacemos split para validación (20% de 80% = 16% del total)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.2, stratify=y_trainval, random_state=42
)

print("Tamaños:")
print("Train:", X_train.shape)
print("Val:  ", X_val.shape)
print("Test: ", X_test.shape)

Tamaños:
Train: (960, 16)
Val:   (240, 16)
Test:  (300, 16)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_base = RandomForestClassifier(
    n_estimators=500, max_depth=None, n_jobs=-1, random_state=42, class_weight="balanced_subsample"
)
rf_base.fit(X_train, y_train)

importances = pd.Series(rf_base.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("Importancia (RF):\n", importances)

# K = número de features top a seleccionar
K = min(6, X_train.shape[1])
topK = importances.index[:K].tolist()
print("Top-K features:", topK)

Importancia (RF):
 M_G                         0.316703
parallax                    0.140474
phot_rp_mean_mag            0.106992
parallax_error              0.088246
phot_g_mean_mag             0.083785
phot_g_mean_flux            0.082242
bp_g                        0.054870
BP_RP                       0.034056
bp_rp                       0.032203
phot_bp_mean_mag            0.024989
g_rp                        0.016989
phot_bp_rp_excess_factor    0.012983
astrometric_excess_noise    0.002129
ruwe                        0.001485
dec                         0.000972
ra                          0.000883
dtype: float64
Top-K features: ['M_G', 'parallax', 'phot_rp_mean_mag', 'parallax_error', 'phot_g_mean_mag', 'phot_g_mean_flux']


In [ ]:
from sklearn.feature_selection import RFE

rf_for_rfe = RandomForestClassifier(
    n_estimators=400, max_depth=None, n_jobs=-1, random_state=7, class_weight="balanced_subsample"
)
# Número de features objetivo
n_select = K  # usa el K que se usó antes
rfe = RFE(estimator=rf_for_rfe, n_features_to_select=n_select, step=1)
rfe.fit(X_train, y_train)

selected_mask = rfe.support_
selected_features = X_train.columns[selected_mask].tolist()
print("RFE selected:", selected_features)

RFE selected: ['M_G', 'parallax', 'parallax_error', 'phot_g_mean_flux', 'phot_g_mean_mag', 'phot_rp_mean_mag']


Comparación de entrenamiento con todas las variables vs K variables

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def eval_model(clf, X_tr, y_tr, X_te, y_te):
    # CV
    acc_cv = cross_val_score(clf, X_tr, y_tr, cv=cv, scoring="accuracy").mean()
    f1_cv  = cross_val_score(clf, X_tr, y_tr, cv=cv, scoring="f1_macro").mean()
    # Fit + test
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    f1  = f1_score(y_te, y_pred, average="macro")
    # ROC-AUC one-vs-rest (requiere predict_proba)
    if hasattr(clf, "predict_proba"):
        y_bin_te = label_binarize(y_te, classes=np.arange(len(le.classes_)))
        y_proba  = clf.predict_proba(X_te)
        auc = roc_auc_score(y_bin_te, y_proba, average="macro", multi_class="ovr")
    else:
        auc = np.nan
    return acc_cv, f1_cv, acc, f1, auc, confusion_matrix(y_te, y_pred)

# Modelo base a comparar
rf_final = RandomForestClassifier(
    n_estimators=600, max_depth=None, n_jobs=-1, random_state=123, class_weight="balanced_subsample"
)

# (A) Todas las variables
metrics_all = eval_model(rf_final, X_train, y_train, X_test, y_test)

# (B) Solo variables seleccionadas por RFE
X_train_sel = X_train[selected_features]
X_test_sel  = X_test[selected_features]
metrics_sel = eval_model(rf_final, X_train_sel, y_train, X_test_sel, y_test)

# (C) Solo variables seleccionadas por RFE Validación
X_val_sel = X_val[selected_features]
metrics_sel_val = eval_model(rf_final, X_train_sel, y_train, X_val_sel, y_val)


print("\n=== Resultados RF (todas) ===")
print(f"CV Acc: {metrics_all[0]:.3f} | CV Macro-F1: {metrics_all[1]:.3f} | Test Acc: {metrics_all[2]:.3f} | Test Macro-F1: {metrics_all[3]:.3f} | Test ROC-AUC: {metrics_all[4]:.3f}")
print("Matriz de confusión (test):\n", metrics_all[5])

print("\n=== Resultados RF (RFE) ===")
print(f"CV Acc: {metrics_sel[0]:.3f} | CV Macro-F1: {metrics_sel[1]:.3f} | Test Acc: {metrics_sel[2]:.3f} | Test Macro-F1: {metrics_sel[3]:.3f} | Test ROC-AUC: {metrics_sel[4]:.3f}")
print("Matriz de confusión (test):\n", metrics_sel[5])

print("\n=== Resultados RF (RFE) Validación ===")
print(f"CV Acc: {metrics_sel_val[0]:.3f} | CV Macro-F1: {metrics_sel_val[1]:.3f} | Test Acc: {metrics_sel_val[2]:.3f} | Test Macro-F1: {metrics_sel_val[3]:.3f} | Test ROC-AUC: {metrics_sel_val[4]:.3f}")
print("Matriz de confusión (test):\n", metrics_sel_val[5])



=== Resultados RF (todas) ===
CV Acc: 0.996 | CV Macro-F1: 0.996 | Test Acc: 1.000 | Test Macro-F1: 1.000 | Test ROC-AUC: 1.000
Matriz de confusión (test):
 [[100   0   0]
 [  0 100   0]
 [  0   0 100]]

=== Resultados RF (RFE) ===
CV Acc: 0.981 | CV Macro-F1: 0.981 | Test Acc: 0.970 | Test Macro-F1: 0.970 | Test ROC-AUC: 0.999
Matriz de confusión (test):
 [[100   0   0]
 [  0  94   6]
 [  0   3  97]]

=== Resultados RF (RFE) Validación ===
CV Acc: 0.981 | CV Macro-F1: 0.981 | Test Acc: 0.967 | Test Macro-F1: 0.967 | Test ROC-AUC: 0.996
Matriz de confusión (test):
 [[80  0  0]
 [ 0 75  5]
 [ 0  3 77]]


In [74]:
dataset = df[selected_features + ["target"]]
dataset

,M_G,parallax,parallax_error,phot_g_mean_flux,phot_g_mean_mag,phot_rp_mean_mag,target
0,2.346488,0.468424,0.012727,47602.451754,13.993294,13.325885,Giant
1,2.462424,0.344142,0.024055,23091.534139,14.778735,14.204826,Giant
2,1.161969,0.849927,0.018639,466575.903749,11.515061,10.861386,Giant
3,0.137227,0.183334,0.017172,55787.957829,13.821015,12.687241,Giant
4,2.447671,0.717614,0.013189,101779.386976,13.168218,12.545147,Giant
...,...,...,...,...,...,...,...
1495,3.051143,0.474276,0.025061,25500.664057,14.670988,14.183872,MS
1496,4.833715,0.482834,0.044930,5117.490488,16.414724,15.856883,MS
1497,11.196386,3.245249,0.193633,659.004582,18.640146,17.356352,MS
1498,8.687364,1.361344,0.093489,1169.330099,18.017525,17.061110,MS


## 3.- Análisis con 3 modelos usando como las variables seleccionadas

In [75]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
# =======================
# 0) Datos desde `dataset`
# =======================
# Asumimos que la columna 'target' existe en `dataset` y el resto son features (X)
X = dataset.drop(columns=['target'])
y = dataset['target']

# Codifica y (MS, Giant, WD -> 0/1/2)
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Escalado (beneficia a KNN y SVM; RF no lo necesita pero tampoco le afecta)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [77]:

# ==========================================
# 1) Partición: Train (64%), Val (16%), Test (20%)
# ==========================================
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_scaled, y_enc, test_size=0.20, stratify=y_enc, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.20, stratify=y_trainval, random_state=42
)

print("Train:", X_train.shape, " Val:", X_val.shape, " Test:", X_test.shape)

Train: (960, 6)  Val: (240, 6)  Test: (300, 6)


In [78]:
# =======================
# 2) Modelos a comparar
# =======================
models = {
    "KNN": KNeighborsClassifier(n_neighbors=15, weights="distance"),
    "RandomForest": RandomForestClassifier(n_estimators=500, random_state=42, class_weight="balanced_subsample", n_jobs=-1),
    "SVM": SVC(kernel="rbf", C=10, gamma="scale", probability=True, random_state=42)
}

In [79]:
# =====================================
# 3) Función de métricas (AUC robusto)
# =====================================
def compute_metrics(clf, X_tr, y_tr, X_ev, y_ev, class_names):
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_ev)
    acc = accuracy_score(y_ev, y_pred)
    f1  = f1_score(y_ev, y_pred, average="macro")

    auc = np.nan
    if hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(X_ev)
        classes_present = np.unique(y_ev)

        if len(classes_present) == 1:
            auc = np.nan  # no se define AUC con una sola clase
        elif len(classes_present) == 2:
            # AUC binario: usar solo la prob de la clase positiva
            pos_class = classes_present[1]
            pos_idx = list(clf.classes_).index(pos_class)
            y_true_bin = (y_ev == pos_class).astype(int)
            auc = roc_auc_score(y_true_bin, proba[:, pos_idx])
        else:
            # AUC multiclase OvR
            y_bin = label_binarize(y_ev, classes=clf.classes_)
            auc = roc_auc_score(y_bin, proba, average="macro", multi_class="ovr")

    cm = confusion_matrix(y_ev, y_pred)
    report = classification_report(y_ev, y_pred, target_names=class_names, zero_division=0)
    return acc, f1, auc, cm, report


In [93]:
# Features (X) y target (y)
X_nan_train = pd.DataFrame(X_train)
X_nan_val = pd.DataFrame(X_val)


# Imputar valores NaN con la media de cada columna
imputer = SimpleImputer(strategy="mean")
X_train = pd.DataFrame(imputer.fit_transform(X_nan_train), columns=X_nan_train.columns)
X_val = pd.DataFrame(imputer.fit_transform(X_nan_val), columns=X_nan_val.columns)

print("NaNs en Train antes:", X_train.isna().sum().sum())
print("NaNs en Train después:", X_train.isna().sum().sum())
print("NaNs en Val antes:", X_val.isna().sum().sum())
print("NaNs en Val después:", X_val.isna().sum().sum())

NaNs en Train antes: 0
NaNs en Train después: 0
NaNs en Val antes: 0
NaNs en Val después: 0


In [94]:
print("NaNs en Train:", pd.DataFrame(X_train).isna().sum().sum())

NaNs en Train: 0


In [95]:
# ============================================
# 4) Comparación en VALIDACIÓN (elegir ganador)
# ============================================
val_rows = []
val_details = {}  # para guardar matrices y reportes

for name, clf in models.items():
    acc, f1, auc, cm, rep = compute_metrics(clf, X_train, y_train, X_val, y_val, le.classes_)
    val_rows.append({"Model": name, "Val_Accuracy": acc, "Val_MacroF1": f1, "Val_ROC_AUC": auc})
    val_details[name] = {"cm": cm, "report": rep}

df_val = pd.DataFrame(val_rows).sort_values("Val_MacroF1", ascending=False)
print("\n=== Validation results ===")
print(df_val)

best_model_name = df_val.iloc[0]["Model"]
best_model = models[best_model_name]
print(f"\nSelected best model on validation: {best_model_name}")
print("\nConfusion matrix (VAL):\n", val_details[best_model_name]["cm"])
print("\nClassification report (VAL):\n", val_details[best_model_name]["report"])


=== Validation results ===
          Model  Val_Accuracy  Val_MacroF1  Val_ROC_AUC
1  RandomForest      0.970833     0.970832     0.994948
0           KNN      0.962500     0.962163     0.990339
2           SVM      0.962500     0.961949     0.989349

Selected best model on validation: RandomForest

Confusion matrix (VAL):
 [[80  0  0]
 [ 0 76  4]
 [ 0  3 77]]

Classification report (VAL):
               precision    recall  f1-score   support

       Giant       1.00      1.00      1.00        80
          MS       0.96      0.95      0.96        80
          WD       0.95      0.96      0.96        80

    accuracy                           0.97       240
   macro avg       0.97      0.97      0.97       240
weighted avg       0.97      0.97      0.97       240



In [96]:
# ===============================================================
# 5) Reentrenar en TRAIN+VAL con el mejor modelo y evaluar en TEST
# ===============================================================
X_trfin = np.vstack([X_train, X_val])
y_trfin = np.hstack([y_train, y_val])

test_acc, test_f1, test_auc, test_cm, test_rep = compute_metrics(
    best_model, X_trfin, y_trfin, X_test, y_test, le.classes_
)

print("\n=== Test results (best model) ===")
print(f"Model: {best_model_name}")
print(f"Test Accuracy:  {test_acc:.3f}")
print(f"Test Macro-F1:  {test_f1:.3f}")
print(f"Test ROC-AUC:   {test_auc:.3f}" if not np.isnan(test_auc) else "Test ROC-AUC:   NA")
print("\nConfusion matrix (TEST):\n", test_cm)
print("\nClassification report (TEST):\n", test_rep)


=== Test results (best model) ===
Model: RandomForest
Test Accuracy:  0.977
Test Macro-F1:  0.977
Test ROC-AUC:   0.999

Confusion matrix (TEST):
 [[100   0   0]
 [  0  93   7]
 [  0   0 100]]

Classification report (TEST):
               precision    recall  f1-score   support

       Giant       1.00      1.00      1.00       100
          MS       1.00      0.93      0.96       100
          WD       0.93      1.00      0.97       100

    accuracy                           0.98       300
   macro avg       0.98      0.98      0.98       300
weighted avg       0.98      0.98      0.98       300

